In [4]:
import pandas as pd
import numpy as np
import sqlite3

In [5]:
def load_db(path):
    conn = sqlite3.connect(path)
    return conn

def get_tables(conn):
    return pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

def load_table(conn, table):
    query = "SELECT * FROM {}".format(table)
    df = pd.read_sql_query(query, conn)
    return df



In [15]:
go_crm = load_db('source/go_crm.sqlite')
go_sales = load_db('source/go_sales.sqlite')

tables_crm = get_tables(go_crm)
tables_sales = get_tables(go_sales)

OperationalError: unable to open database file